# DMA v2 — Daily Entry Screener (After Hours)
Scans a list of tickers **after market close** and writes `signals/YYYY-MM-DD/entries.csv` for trades to take **next day**.

**Entry rule (v2):**
- 10DMA crosses **above** 20DMA from below, and
- Then two consecutive **green** candles, and
- The **second** green candle closes **above** the first.

**Notes**
- Uses Yahoo Finance daily data, converted to `Asia/Kolkata`.
- Optional filters (price>200DMA, slope>0, RSI, ADX, ATR band, volume surge).
- No plotting. Fully parameterized at the top.


## Parameters (edit me)

In [1]:

# ====================== USER PARAMETERS ======================
TICKERS = ['SBILIFE.NS']



START_DATE = "2025-07-31"
END_DATE   = None
TZ = "Asia/Kolkata"

# Core DMAs
FAST_DMA = 10
SLOW_DMA = 20
REGIME_DMA = 200  # for optional regime filter

# Cross timing (relative to the second green candle at time t)
REQUIRE_CROSS_EXACTLY_T_MINUS_2 = True
CROSS_WINDOW_AFTER = 3  # used only if above flag is False

# Optional Filters
USE_FILTER_PRICE_ABOVE_200DMA = False
USE_FILTER_SLOW_SLOPE_POSITIVE = False
USE_FILTER_RSI = True; RSI_LEN = 14; RSI_MIN = 52
USE_FILTER_ADX = True; ADX_LEN = 14; ADX_MIN = 20
USE_FILTER_VOLUME_SURGE = True; VOL_LOOKBACK = 20; VOL_FACTOR = 1.2
USE_FILTER_ATR_BAND = True; ATR_LEN = 14; ATR_MIN_PCT = 0.6; ATR_MAX_PCT = 3.5

# Output folder base
OUTPUT_BASE = "signals"
# ============================================================


## Install & Imports

In [2]:

# !pip install yfinance pandas numpy pytz python-dateutil

import os, pytz
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime


## Helpers: Data, Indicators, Signals

In [3]:

def _to_tz_local_naive(df, tz):
    if df.index.tz is None:
        df.index = df.index.tz_localize('UTC').tz_convert(tz).tz_localize(None)
    else:
        df.index = df.index.tz_convert(tz).tz_localize(None)
    return df

def fetch_1d(ticker, start, end, tz):
    df = yf.download(ticker, start=start, end=end, interval='1d', auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty: return df
    df = df.rename(columns=str.title)
    df = _to_tz_local_naive(df, tz).dropna(subset=['Open','High','Low','Close','Volume'])
    return df

def SMA(s, n): return s.rolling(n, min_periods=n).mean()

def RSI(close, n=14):
    d = close.diff()
    up = d.clip(lower=0)
    dn = -d.clip(upper=0)
    avg_up = up.ewm(alpha=1/n, adjust=False).mean()
    avg_dn = dn.ewm(alpha=1/n, adjust=False).mean()
    rs = avg_up / avg_dn.replace(0, np.nan)
    return (100 - 100/(1+rs)).fillna(50)

def ATR(high, low, close, n=14):
    pc = close.shift(1)
    tr = pd.concat([(high-low), (high-pc).abs(), (low-pc).abs()], axis=1).max(axis=1)
    return tr.ewm(alpha=1/n, adjust=False).mean()

def ADX(high, low, close, n=14):
    up = high.diff(); dn = -low.diff()
    plus_dm  = np.where((up>dn) & (up>0), up, 0.0)
    minus_dm = np.where((dn>up) & (dn>0), dn, 0.0)
    pc = close.shift(1)
    tr = pd.concat([(high-low), (high-pc).abs(), (low-pc).abs()], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/n, adjust=False).mean()
    plus_di = 100 * pd.Series(plus_dm, index=close.index).ewm(alpha=1/n, adjust=False).mean() / atr
    minus_di= 100 * pd.Series(minus_dm,index=close.index).ewm(alpha=1/n, adjust=False).mean() / atr
    dx = ( (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0,np.nan) ) * 100
    return dx.ewm(alpha=1/n, adjust=False).mean()

def add_indicators(df, fast=10, slow=20, regime=200, rsi_len=14, adx_len=14, atr_len=14):
    o = df.copy()
    o[f'DMA_{fast}'] = SMA(o['Close'], fast)
    o[f'DMA_{slow}'] = SMA(o['Close'], slow)
    o[f'DMA_{regime}'] = SMA(o['Close'], regime)
    o['Green'] = o['Close'] > o['Open']
    o['Red']   = o['Close'] < o['Open']
    o['RSI'] = RSI(o['Close'], rsi_len)
    o['ATR'] = ATR(o['High'], o['Low'], o['Close'], atr_len)
    o['ATR_PCT'] = o['ATR'] / o['Close'] * 100
    o['ADX'] = ADX(o['High'], o['Low'], o['Close'], adx_len)
    o['SLOW_SLOPE'] = o[f'DMA_{slow}'].diff()
    o['VOL_MA'] = SMA(o['Volume'], 20)
    return o

def mark_crosses(out, fast=10, slow=20):
    f, s = f'DMA_{fast}', f'DMA_{slow}'
    out['CrossUp'] = (out[f].shift(1) <= out[s].shift(1)) & (out[f] > out[s])
    out['CrossDn'] = (out[f].shift(1) >= out[s].shift(1)) & (out[f] < out[s])
    return out

def entry_signal(o, fast=10, slow=20,
                 exact_t_minus_2=True, cross_window_after=3,
                 regime_ok=True, slope_ok=True, rsi_ok=True, rsi_min=50,
                 adx_ok=True, adx_min=18, vol_surge=False, vol_lb=20, vol_factor=1.2,
                 atr_band=True, atr_min_pct=0.5, atr_max_pct=4.0):
    two_green = (o['Green'] & o['Green'].shift(1))
    higher_close = o['Close'] > o['Close'].shift(1)
    base = two_green & higher_close

    if exact_t_minus_2:
        cross_cond = o['CrossUp'].shift(2).fillna(False)
    else:
        c = False
        for k in range(2, 2 + cross_window_after):
            c = c | o['CrossUp'].shift(k).fillna(False)
        cross_cond = c

    cond = base & cross_cond

    if regime_ok: cond &= o['Close'] > o['DMA_200']
    if slope_ok:  cond &= o['SLOW_SLOPE'] > 0
    if rsi_ok:    cond &= o['RSI'] >= rsi_min
    if adx_ok:    cond &= o['ADX'] >= adx_min
    if vol_surge: cond &= o['Volume'] >= (o['VOL_MA'] * vol_factor)
    if atr_band:  cond &= (o['ATR_PCT'] >= atr_min_pct) & (o['ATR_PCT'] <= atr_max_pct)
    return cond.fillna(False)

def exit_signal(o, slow=20, protective_on_close_below_slow=True):
    red_seq = (o['Red'] & o['Red'].shift(1))
    below_slow = o['Close'] < o[f'DMA_{slow}']
    sig = (red_seq & below_slow)
    if protective_on_close_below_slow:
        sig = sig | below_slow
    return sig.fillna(False)

def india_market_open_now(ts=None, tz=TZ):
    import pytz
    IST = pytz.timezone(tz)
    now = ts or datetime.now(IST)
    if now.weekday() >= 5:
        return False
    start = now.replace(hour=9, minute=15, second=0, microsecond=0)
    end   = now.replace(hour=15, minute=30, second=0, microsecond=0)
    return start <= now <= end

def last_completed_index(df, tz=TZ):
    if df.empty: return None
    if india_market_open_now(tz=tz):
        return df.index[-2] if len(df)>=2 else None
    return df.index[-1]


## Run screener & save entries.csv

In [4]:

IST = pytz.timezone(TZ)
run_date = datetime.now(IST).strftime("%Y-%m-%d")
out_dir = os.path.join(OUTPUT_BASE, run_date)
os.makedirs(out_dir, exist_ok=True)

rows = []
for t in TICKERS:
    print(f"Scanning {t} ...")
    df = fetch_1d(t, START_DATE, END_DATE, TZ)
    if df.empty:
        print(f"[WARN] {t}: no data"); 
        continue
    df = add_indicators(df, FAST_DMA, SLOW_DMA, REGIME_DMA, RSI_LEN, ADX_LEN, ATR_LEN)
    df = mark_crosses(df, FAST_DMA, SLOW_DMA)
    ent = entry_signal(
        df, FAST_DMA, SLOW_DMA,
        REQUIRE_CROSS_EXACTLY_T_MINUS_2, CROSS_WINDOW_AFTER,
        USE_FILTER_PRICE_ABOVE_200DMA, USE_FILTER_SLOW_SLOPE_POSITIVE,
        USE_FILTER_RSI, RSI_MIN,
        USE_FILTER_ADX, ADX_MIN,
        USE_FILTER_VOLUME_SURGE, VOL_LOOKBACK, VOL_FACTOR,
        USE_FILTER_ATR_BAND, ATR_MIN_PCT, ATR_MAX_PCT
    )
    idx = last_completed_index(df, TZ)
    if idx is None:
        continue
    if bool(ent.loc[idx]):
        # candidate for next day
        rows.append({
            "Ticker": t,
            "SignalDate": idx.strftime("%Y-%m-%d"),
            "NextTradeDate_hint": (idx + pd.Timedelta(days=1)).strftime("%Y-%m-%d"),
            "Close": float(df.loc[idx, "Close"]),
            f"DMA_{FAST_DMA}": float(df.loc[idx, f"DMA_{FAST_DMA}"]),
            f"DMA_{SLOW_DMA}": float(df.loc[idx, f"DMA_{SLOW_DMA}"]),
            "RSI": float(df.loc[idx, "RSI"]) if "RSI" in df.columns else None,
            "ADX": float(df.loc[idx, "ADX"]) if "ADX" in df.columns else None,
            "ATR_PCT": float(df.loc[idx, "ATR_PCT"]) if "ATR_PCT" in df.columns else None
        })

entries_df = pd.DataFrame(rows)
out_csv = os.path.join(out_dir, "entries.csv")
entries_df.to_csv(out_csv, index=False)
print("\nSaved entries to:", out_csv)
print(entries_df.to_string(index=False) if not entries_df.empty else "No entries today.")


Scanning SBILIFE.NS ...

Saved entries to: signals/2025-09-20/entries.csv
No entries today.


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_19290/1704235443.py:74: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)
